<a href="https://colab.research.google.com/github/denalimarsh/Deep-Learning/blob/main/Keras_building_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------- Building a dense layer -------------
from tensorflow import keras

class SimpleDense(keras.layers.Layer): # All Keras layers inherit from the base Layers class
  def __init__(self, units, activation=None):
    super().__init__()
    self.units = units
    self.activation = activation
  
  # build() creates weights
  def build(self, input_shape):
    input_dim = input_shape[-1]

    # add_weight is a shortcut method for creating weights, it is the same as:
    # self.W = tf.Variable(tf.random.uniform(w_shape))
    self.W = self.add_weight(shape=(input_dim, self.units), initializer="random_normal")
    self.b = self.add_weight(shape=(self.units,), initializer="zeros")

  # call() executes the forward pass. In pseudocode, it looks like this:
  #
  # if not self.built:
  #  self.build(inputs.shape)
  #  self.built = True
  # return self.call(inputs)
  #

  def call(self, inputs):
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None:
      y = self.activation(y)
    return y

# ------------- Keras dense layers -------------

from tensorflow.keras import layers
layer = layers.Dense(32, activation="relu") # A dense layer with 32 output units

In [ ]:
# ------------- Keras linear classifier -------------
model = keras.Sequential([keras.layers.Dense(1)])

# compile configures the training process
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])
# It's possible to use shortcuts for optimizer, loss, and metrics parameters:
# model.compile(optimizer="rmsprop", loss="mean_squared_error", metrics=["accuracy"])

# fit implements the training loop
history = model.fit(
    inputs,        # Input samples as NumPy array
    targets,       # The corresponding training targets as NumPy array
    epochs=5,      # Training loop will iterate over the data 5 times
    batch_size=128 # The training loop will iterate over the data in batches of 128 samples
)
print(history.history)

In [ ]:
# ------------- Using validation data as a control group -------------
from tensorflow import keras

model = keras.Sequential([keras.layers.Dense(1)]) # Define a model with one Dense Layer
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1), # Define model's training strategy
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

# Shuffle inputs/target labels so validation data samples aren't from a single class of input data
indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permuation]

# Reserve 30% of training inputs/targets for validation
num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

model.fit(
    training_inputs,  # Training data used to update model weights
    training_targets, # Training data used to update model weights
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets) # Validation data used to monitor the loss and metrics
)

# model.evaluate() can be used to compute validation loss and metirics after the training is complete
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128) 

# ------------- Inference: using a model after training -------------

# predict() iterates over small batches of data and returns a NumPy array of predictions
predictions = model.predict(new_inputs, batch_size=128)
print(predictions[:10])